<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##試算表個人記帳
- 可自行加入日期時間或按enter輸入現在時間。
- 輸入品項後可選擇屬於哪個種類，用於分析圖表。
- 可計算出當日總額即當月總額。
- [試算表連結](https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0)

In [74]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [75]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14qT33yoGpMBe868XI6yvd8Hy7DldM4RhovGYNRtCUMk/edit?hl=zh-tw&gid=0#gid=0')

In [76]:
worksheet = gsheets.worksheet("工作表1")
headers = ["日期", "時間", "品項", "種類", "金額"]
first_row = worksheet.row_values(1)
if not first_row or first_row != headers:
    worksheet.clear()
    worksheet.append_row(headers)

In [78]:
 from datetime import datetime
 date_str = input("請輸入日期 (格式：YYYY-MM-DD, Enter=今天): ")
if date_str.strip() == "":
    date_str = datetime.now().strftime("%Y-%m-%d")
else:
    datetime.strptime(date_str, "%Y-%m-%d")

# 時間
time_str = input("請輸入時間 (格式：HH:MM, Enter=現在時間): ")
if time_str.strip() == "":
    time_str = datetime.now().strftime("%H:%M")
else:
    datetime.strptime(time_str, "%H:%M")

# 品項
item = input("請輸入品項: ")

# 金額
amount = float(input("請輸入金額: "))

# 類別選單
categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]
print("請選擇種類：")
for i, c in enumerate(categories, 1):
    print(f"{i}. {c}")
choice = int(input("輸入數字選擇: "))
category = categories[choice - 1]
worksheet.append_row([date_str, time_str, item, category, amount], value_input_option="USER_ENTERED")
print("✅ 新增成功！")

請輸入日期 (格式：YYYY-MM-DD, Enter=今天): 
請輸入時間 (格式：HH:MM, Enter=現在時間): 
請輸入品項: 蛋糕
請輸入金額: 20
請選擇種類：
1. 早餐
2. 午餐
3. 晚餐
4. 點心
5. 飲料
6. 衣服
7. 娛樂
8. 交通
輸入數字選擇: 4
✅ 新增成功！


In [79]:
# === 轉 DataFrame 處理 ===
sheets = worksheet.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")

today_str = datetime.now().strftime("%Y-%m-%d")

In [80]:

## === 輸入想要加總的日期 ===
date_str = input("請輸入要結算的日期 (格式：YYYY-MM-DD, Enter=今天): ")
if date_str.strip() == "":
    date_str = datetime.now().strftime("%Y-%m-%d")
else:
    datetime.strptime(date_str, "%Y-%m-%d")  # 驗證格式

# === 計算該日期的總支出 ===
df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")

# 找出該日的所有資料（排除「當天總支出」）
target_data = df[(df["日期_str"] == date_str) & (df["品項"] != "當天總支出")]
day_total = target_data["金額"].sum()

# === 找該日最後一筆資料的位置 ===
rows = worksheet.findall(date_str)
last_row_day = rows[-1].row if rows else len(worksheet.get_all_values())

# === 檢查下一行是否已有「當天總支出」 ===
sheet_rows = len(worksheet.get_all_values())
next_row_values = []
if last_row_day + 1 <= sheet_rows:
    next_row_values = worksheet.row_values(last_row_day + 1)

if next_row_values and "當天總支出" in next_row_values:
    # ✅ 已經有 → 更新金額
    worksheet.update_cell(last_row_day + 1, 5, day_total)
    target_row = last_row_day + 1
else:
    # ✅ 沒有 → 新增一列
    worksheet.append_row([date_str, "", "當天總支出", "", day_total],
                         value_input_option="USER_ENTERED")
    target_row = len(worksheet.get_all_values())

# === 標記顏色（黃色底 + 粗體） ===
fmt = {
    "backgroundColor": {"red": 1, "green": 1, "blue": 0},  # 黃色
    "textFormat": {"bold": True}
}
worksheet.format(f"A{target_row}:E{target_row}", fmt)

print(f"📌 {date_str} 的總支出已更新：{day_total} 元")



請輸入要結算的日期 (格式：YYYY-MM-DD, Enter=今天): 
📌 2025-09-11 的總支出已更新：60.0 元


In [81]:
# === 每月總結 ===
year = int(date_str[:4])
month = int(date_str[5:7])
month_total = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]["金額"].sum()
print(f"📌 {date_str[:7]} 當月總支出：{month_total} 元")

📌 2025-09 當月總支出：60.0 元


In [82]:
from datetime import datetime
import calendar

# === 更新圓餅圖 (隨時更新) ===
def update_pie_chart_direct(worksheet, year, month, last_row):
    chart_request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": f"{year}-{month:02d} 各種類支出分布",
                    "pieChart": {
                        "legendPosition": "RIGHT_LEGEND",
                        "domain": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 1,           # 從第2列開始 (跳過標題)
                                    "endRowIndex": last_row,      # 到最後一列
                                    "startColumnIndex": 2,        # C 欄 (種類)
                                    "endColumnIndex": 3
                                }]
                            }
                        },
                        "series": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 1,
                                    "endRowIndex": last_row,
                                    "startColumnIndex": 4,        # E 欄 (金額)
                                    "endColumnIndex": 5
                                }]
                            }
                        },
                        "pieHole": 0.3
                    }
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {"sheetId": worksheet.id, "rowIndex": 0, "columnIndex": 9}
                    }
                }
            }
        }
    }
    gsheets.batch_update({"requests": [chart_request]})
    print("✅ 圓餅圖 (直接用原始資料區) 已更新")

    # === 每次輸入資料後都更新圓餅圖 ===
year = int(date_str[:4])
month = int(date_str[5:7])
update_pie_chart(df, worksheet, year, month)

/tmp/ipython-input-690839910.py:10: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("G1", [["種類", "總金額"]])
/tmp/ipython-input-690839910.py:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update("G2", cat_summary.values.tolist())


✅ 圓餅圖已更新


In [83]:
# === 生成折線圖 (月底觸發) ===
def generate_line_chart(df, worksheet, year, month):
    month_data = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]
    day_summary = month_data.groupby("日期_str")["金額"].sum().reset_index()

    # 寫入 J 欄
    worksheet.update("J1", [["日期", "總支出"]])
    if not day_summary.empty:
        worksheet.update("J2", day_summary.values.tolist())

    # 建立折線圖
    chart_request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": f"{year}-{month:02d} 每日總支出折線圖",
                    "basicChart": {
                        "chartType": "LINE",
                        "legendPosition": "BOTTOM_LEGEND",
                        "axis": [
                            {"position": "BOTTOM_AXIS", "title": "日期"},
                            {"position": "LEFT_AXIS", "title": "支出金額"}
                        ],
                        "domains": [{
                            "domain": {
                                "sourceRange": {
                                    "sources": [{"sheetId": worksheet.id,
                                                 "startRowIndex": 0,
                                                 "endRowIndex": len(day_summary)+1,
                                                 "startColumnIndex": 9,  # J
                                                 "endColumnIndex": 10}]
                                }
                            }
                        }],
                        "series": [{
                            "series": {
                                "sourceRange": {
                                    "sources": [{"sheetId": worksheet.id,
                                                 "startRowIndex": 0,
                                                 "endRowIndex": len(day_summary)+1,
                                                 "startColumnIndex": 10,  # K
                                                 "endColumnIndex": 11}]
                                }
                            }
                        }]
                    }
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {"sheetId": worksheet.id, "rowIndex": 20, "columnIndex": 9}
                    }
                }
            }
        }
    }
    gsheets.batch_update({"requests": [chart_request]})
    print("📊 折線圖已生成 (月底)")
    # === 如果是月底，生成折線圖 ===
day = int(date_str[8:10])
last_day = calendar.monthrange(year, month)[1]
if day == last_day:
    generate_line_chart(df, worksheet, year, month)